< [Prepare the initial section of the model](STREAM_Initial.ipynb) | [Contents](Contents.ipynb) >

# 4. Building the dynamic model
In the previous section we have prepared the `initial` section of the STREAM model. I.e. we loaded the static maps, defined the constants and we created input rasters using lookup tables.

Now we're going to build the dynamic model in the `dynamic` section.

## 4.1 Interception
Previously we have already added the code to read the precipitation for each time step.
Now we need to calculate amount of interception. 
The amount of interception is the amount of precipitation, but limited to the land-use dependent interception threshold.

Therefore we add the following line to the code:

```Python
Interception = min(Precipitation, self.InterceptionThreshold)  
```

The PCRaster [min](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_min.html) operation is used here to get the minimum value of `Preciptiation` and `self.InterceptionThreshold` for each cell.

Let's also add a line to report the amount of `Interception` as a PCRaster map stack (temporal map series).

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Add here the calculation of the flow direction map
        self.flowdirection = lddcreate(DEM,1e31,1e31,1e31,1e31)
        self.flowdirection = lddmask(self.flowdirection,self.mask)
        self.report(self.flowdirection,"./Data/ldd")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Now we can use Aguila from the command prompt to compare the precipitation and interception time series.
Go to the command prompt, change the directory to the data and type at the prompt:
```
aguila --timesteps [1,10,1] pr int d
```

With `[1,10,1]` we indicate first time step, last time step and interval respectively.
Check the results.

## 4.2 Net precipitation
The net precipitation is the amount of precipitation minus the interception.

Add the code to line 56 and use NetPrecipitation as output variable. Also add a line to write the result to disk with he file name `pn`.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Add here the calculation of the flow direction map
        self.flowdirection = lddcreate(DEM,1e31,1e31,1e31,1e31)
        self.flowdirection = lddmask(self.flowdirection,self.mask)
        self.report(self.flowdirection,"./Data/ldd")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation

        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Run the model and visualise the result of `NetPrecipitation` using Aguila at the command prompt.

## 4.3 Evapotranspiration
The actual evapotranspiration is available from meteorological stations in the study area. The locations of the stations are on a map that we have read in the `initial` section: `self.metstat`.

Visualise `metstat.map` with Aguila from the command prompt. 
* How many meteorological stations are there in the study area?
* What is the data type of this map?

The evapotranspiration data is stored in the [PCRaster time series](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/secdatbase.html#time-series-format) format in the file `et.tss`.
Let's have a look at the file (you can open it in Notepad for example).

```
"ET,time series"			
4			
time			
station 1			
station 2			
station 3			
1	6.152	5.231	5.605
2	6.002	5.324	5.528
3	4.165	3.507	3.749
...
```

The first line of the file contains the title.
The second line has the amount of columns.
The following rows have the column names. The first one is always `time`. The other ones correspond with the station numbers in `metstat.map`: column 2 has the values for station 1, column 3 for station 2, etc. Each row has the values for a time step.

With the PCRaster time series file and the corresponding map with meteorological stations we can create a map with evapotranspiration for each station at each time step using the [`timeinput...`](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_timeinput....html) operation. Because our data results in scalar maps, the operation is `timeinputscalar`.

We add therefore add the following line to the script:
```Python
ETStations = timeinputscalar("./Data/et.tss",self.metstat)
```
We also report the result to disk. While developing a model it is recommended to save results to disk to check if the result is what we expected. Later we can comment out those line to make the calculation faster.

*Note that we now read the flow direction map from disk to save time*

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
        # Create maps with actual evapotranspiration
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
                
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Run the model and check the result with Aguila from the command line.

Having evapotranspiration only at the meteorological stations is not so useful in our spatially explicit model. We therefore have to interpolate the points to cover the whole study area.

Previously you have learned to interpolate using Thiessen polygons and IDW. Add the code to line 60 and call the output `ET`. In line 61 report the result to disk.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.    
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
        # Create maps with actual evapotranspiration
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpolate ETa with IDW

        
                
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Run the model and check the result with Aguila from the command prompt.

* Can you modify the code to do the interpolation with Thiessen polygons?
* What would be the best to use given the amount and distribution of the meteorological stations?

## 4.4 Unsaturated zone

Next, the net precipitation separated into infiltration into the unsaturated and saturated zone using a separation coefficient.
In the `initial` section we have already created a map `SeparationCoefficient` using a lookup table with the land-use map. We have also defined an initial soil moisture storage `self.Su` (mm). To calculate the new soil moisture storage we add the following code:
```Python
self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
```

The unsaturated zone is filled up until field capacity, `SuMax` (mm) is reached. The soil moisture excess, `SuExcess` (mm) goes to the saturated zone with delay factor of `Ku`. This is the only exchange between the unsaturated and saturated zone. In code:
```Python
SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
```

Then the new soil moisture storage in the unsaturated zone can be calculated:
```Python
self.Su = self.Su - SuExcess
```

And we also have subtract the evapotranspiration:
```Python
self.Su = self.Su - ET
```

Let's add the code to lines 64-68 and write the result to disk.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
         
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
        # Create maps with actual evapotranspiration
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpolate ETa with IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modelling of the unsaturated zone

        
        
        
        
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Check the result by visualising `su` with Aguila from the command prompt.

## 4.5 Saturated zone

The saturated zone can be modelled as the sum of the storage of the saturated zone, the amount of net precipitation that infiltrates and the excess of the unsaturated zone:

```Python
self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
```

Let's add this to the script in line 71. Report the result in line 72.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
        # Create maps with actual evapotranspiration
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpolate ETa with IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modelling of the unsaturated zone
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modelling of the saturated zone

              
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Run the model and check the result with Aguila in the command prompt.

## 4.6 Saturated overland flow
There is however a maximum to the storage of the saturated zone `SsMax` (mm). Saturated groundwater flow occurs when storage of the saturated zone `Ss` exceeds `SsMax`. This normally occurs when the groundwater table reaches the surface under saturated soil condition. The situation whereby the groundwater table reaches the surface is determined by the DEM and the bottom of the draining river `Ss0` (m). `SsDEM` (m) is defined as the distance between the surface and the bottom of the draining river.
```Python
SsDEM = DEM + self.Ss0
```

Due to porosity, `SsDEM` is not completely available for water. Therefore we use the following empirical relation:
```Python
self.SsMax = 25 * ln(SsDEM)
```

These equations are not time dependent. Therefore they should be added to the `initial` section. Add the code to line 47 and 48 and report the result in line 49.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculate maximum to the storage of the saturated zone

        
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
        # Create maps with actual evapotranspiration
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpolate ETa with IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modelling of the unsaturated zone
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modelling of the saturated zone
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Now we can calculate the saturated overland flow as the amount that exceeds the storage of the saturated zone:
```Python
SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
```
This condition reads: if the saturated zone storage is larger than the maximum storage, then the saturated overland flow equals the difference between the saturated zone storage and the maximum storage. Else the saturated overland flow is zero.

After calculating the amount of saturated overland flow, we need to update the storage of the saturated zone:
```Python
self.Ss = self.Ss - SaturatedOverlandFlow
```

Let's add this to the script in lines 81 and 82. Report the result in line 83.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculate maximum to the storage of the saturated zone
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
        # Create maps with actual evapotranspiration
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpolate ETa with IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modelling of the unsaturated zone
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modelling of the saturated zone
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        # Calculate the saturated overland flow

        
        
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Run the model and check the result with Aguila in the command prompt.

## 4.7 Quick flow
The quick groundwater flow component is groundwater flow through macropores and cracks. In the STREAM model we calculate the threshold for the quick flow `SsQuick` by multiplying the maximum saturated zone storage `self.SsMax` with the quick flow coefficient `self.QuickFlowCoefficient`, which we derived from the soil map and a lookup table in the `initial` section.
```Python
SsQuick = self.SsMax * self.QuickFlowCoefficient
```
Then the actual quick flow can be calculated as follows:
```Python
QuickFlow = max((self.Ss - SsQuick), 0) / self.rtq
```
`self.rtq` is the recession constant of the quick flow that we already defined in the `initial` section.

Finally we also need to update the saturated zone storage by removing the quick flow:
```Python
self.Ss = self.Ss - QuickFlow
```

Add this to the code below at line 86 and further. Report what is needed.


In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculate maximum to the storage of the saturated zone
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
        # Create maps with actual evapotranspiration
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpolate ETa with IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modelling of the unsaturated zone
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modelling of the saturated zone
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        # Calculate the saturated overland flow
        SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
        self.Ss = self.Ss - SaturatedOverlandFlow
        self.report(SaturatedOverlandFlow,"./Data/saof")
        
        # Calculate quick flow

        
        
        
                             
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

## 4.8 Slow flow
The slow flow depends on the saturated zone storage (`self.Ss`) and is divided by the recession coefficient for the slow flow (`self.rts`) that we defined before in the `initial` section:
```Python
SlowFlow = max(self.Ss, 0) / self.rts
```
Also here we need to update the saturated zone storage:
```Python
self.Ss = self.Ss - SlowFlow
```
Let's also write the `SlowFlow` to disk.
Add these lines to the code at line 92.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculate maximum to the storage of the saturated zone
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
        # Create maps with actual evapotranspiration
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpolate ETa with IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modelling of the unsaturated zone
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modelling of the saturated zone
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        # Calculate the saturated overland flow
        SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
        self.Ss = self.Ss - SaturatedOverlandFlow
        self.report(SaturatedOverlandFlow,"./Data/saof")
        
        # Calculate quick flow
        SsQuick = self.SsMax * self.QuickFlowCoefficient
        QuickFlow = max((self.Ss - SsQuick), 0) / self.rtq
        self.Ss = self.Ss - QuickFlow
        self.report(QuickFlow, "./Data/qflo")  
        
        # Calculate slow flow

        
        
        
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

## 4.9 Capillary rise
The capilary rise from the saturated zone to the unsaturated zone is defined by a minimum and maximum value.
The minimum value `MinCapRise` is a quarter of the maximum storage of the saturated zone.

Add this to line 52 of the `initial` section below, because it is time independent:

```Python
self.MinCapRise = self.SsMax / 4.0
```

If the storage of the saturated zone is larger than the minimum capillary rise, then the capilary rise is the minimum of the amount available in the saturated zone storage, the evapotranspiration and the maximum capillary rise:
```Python
CapRise = ifthenelse(self.Ss > self.MinCapRise, min(self.MaxCapRise,ET,self.Ss), self.MinCapRise)
```
Add this to line 100 below.

Now we also need to update saturated and unsaturated zone storages.

```Python
self.Ss = self.Ss - CapRise
self.Su = self.Su + CapRise
```

Add this to lines 101 and 102 below and report `self.Ss` and `self.Su`. 
Run the model below and check the results.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculate maximum to the storage of the saturated zone
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
        # Minimum Capillary rise

        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
        # Create maps with actual evapotranspiration
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpolate ETa with IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modelling of the unsaturated zone
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modelling of the saturated zone
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        # Calculate the saturated overland flow
        SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
        self.Ss = self.Ss - SaturatedOverlandFlow
        self.report(SaturatedOverlandFlow,"./Data/saof")
        
        # Calculate quick flow
        SsQuick = self.SsMax * self.QuickFlowCoefficient
        QuickFlow = max((self.Ss - SsQuick), 0) / self.rtq
        self.Ss = self.Ss - QuickFlow
        self.report(QuickFlow, "./Data/qflo")  
        
        # Calculate slow flow
        SlowFlow = max(self.Ss, 0) / self.rts
        self.Ss = self.Ss - SlowFlow
        self.report(SlowFlow,"./Data/sflo")
        
        # Capillary rise

        
        
        
        
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

## 4.10 Discharge modelling
The total runoff is the sum of the saturated overland flow, quick flow and slow flow:
```Python
Runoff = SaturatedOverlandFlow + QuickFlow + SlowFlow
```
With the PCRaster [accuflux](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_accuflux.html) operation we can accumulate the runoff amount over the flow direction map and calculate the discharge:
```Python
Discharge = accuflux(self.flowdirection, Runoff) * self.ConvConst
```

This approach can be used when we can assume that all water reaches the outlet in one time step.
Note that we multiply with a conversion factor to have the correct units.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculate maximum to the storage of the saturated zone
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
        # Minimum Capillary rise
        self.MinCapRise = self.SsMax / 4.0
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
        # Create maps with actual evapotranspiration
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpolate ETa with IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modelling of the unsaturated zone
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modelling of the saturated zone
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        # Calculate the saturated overland flow
        SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
        self.Ss = self.Ss - SaturatedOverlandFlow
        self.report(SaturatedOverlandFlow,"./Data/saof")
        
        # Calculate quick flow
        SsQuick = self.SsMax * self.QuickFlowCoefficient
        QuickFlow = max((self.Ss - SsQuick), 0) / self.rtq
        self.Ss = self.Ss - QuickFlow
        self.report(QuickFlow, "./Data/qflo")  
        
        # Calculate slow flow
        SlowFlow = max(self.Ss, 0) / self.rts
        self.Ss = self.Ss - SlowFlow
        self.report(SlowFlow,"./Data/sflo")
        
        # Capillary rise
        CapRise = ifthenelse(self.Ss > self.MinCapRise, min(self.MaxCapRise,ET,self.Ss), self.MinCapRise)
        self.Ss = self.Ss - CapRise
        self.Su = self.Su + CapRise
        self.report(self.Su,"./Data/su")
        self.report(self.Ss,"./Data/ss")
        
        # Calculate total runoff and discharge
        Runoff = SaturatedOverlandFlow + QuickFlow + SlowFlow
        Discharge = accuflux(self.flowdirection, Runoff) * self.ConvConst
        self.report(Discharge,"./Data/q")
        self.report(Runoff,"./Data/runoff")
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Visualise the result with Aguila from the command prompt:
```
aguila --timesteps [1,10,1] q pr
```
With aguila you can also visualise graphs of a selected pixel. Click right on the legend and choose *Show time series...*
* Is there a relation between rainfall and river discharge?

## Calibration and evaluation
After creating the model, you want to compare the results with measurements in order to (1) calibrate model parameters and (2) evaluate the model performance.

With PCRaster we can create time series tables at selected points. Earlier we used `col2map` to convert coordinates of locations to a map. Here we have created a map with four measurement locations at interesting points in the river.

Use Aguila from the command prompt to visualise `measurements.map` together with `q` to find out where the points are.

To report the discharge at those points we need to initialise this in the `initial` section of the model by adding these three lines:
```Python
self.Measurements = self.readmap("./Data/measurements") # read map with measurement locations
DischargeAtMeasurementLocations = "./Data/discharge.tss" # define the output timeseries file name
self.DischargeTSS = TimeoutputTimeseries(DischargeAtMeasurementLocations, self, self.Measurements, noHeader=False)
```

In the `dynamic` section we can then report the values by adding this line:
```Python
self.DischargeTSS.sample(Discharge)
```
Where `self.DischargeTSS` was defined in the `initial` section and we use `Discharge` in brackets to report the values of discharge.

The code should now look like this:

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        self.Ss0 = scalar(100.0)               # Distance between the surface and the bottom of the river.
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculate maximum to the storage of the saturated zone
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
        # Minimum Capillary rise
        self.MinCapRise = self.SsMax / 4.0
        
        # initialise time series output
        self.Measurements = self.readmap("./Data/measurements") # read map with measurement locations
        DischargeAtMeasurementLocations = "./Data/discharge.tss" # define the output timeseries file name
        self.DischargeTSS = TimeoutputTimeseries(DischargeAtMeasurementLocations, 
                                                 self, 
                                                 self.Measurements, 
                                                 noHeader=False)
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculate Net Precipitation
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
        # Create maps with actual evapotranspiration
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpolate ETa with IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modelling of the unsaturated zone
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modelling of the saturated zone
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        # Calculate the saturated overland flow
        SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
        self.Ss = self.Ss - SaturatedOverlandFlow
        self.report(SaturatedOverlandFlow,"./Data/saof")
        
        # Calculate quick flow
        SsQuick = self.SsMax * self.QuickFlowCoefficient
        QuickFlow = max((self.Ss - SsQuick), 0) / self.rtq
        self.Ss = self.Ss - QuickFlow
        self.report(QuickFlow, "./Data/qflo")  
        
        # Calculate slow flow
        SlowFlow = max(self.Ss, 0) / self.rts
        self.Ss = self.Ss - SlowFlow
        self.report(SlowFlow,"./Data/sflo")
        
        # Capillary rise
        CapRise = ifthenelse(self.Ss > self.MinCapRise, min(self.MaxCapRise,ET,self.Ss), self.MinCapRise)
        self.Ss = self.Ss - CapRise
        self.Su = self.Su + CapRise
        self.report(self.Su,"./Data/su")
        self.report(self.Ss,"./Data/ss")
        
        # Calculate total runoff and discharge
        Runoff = SaturatedOverlandFlow + QuickFlow + SlowFlow
        Discharge = accuflux(self.flowdirection, Runoff) * self.ConvConst
        self.report(Discharge,"./Data/q")
        self.report(Runoff,"./Data/runoff")
        
        # Report discharge time series at measurement locations
        self.DischargeTSS.sample(Discharge)
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

We can use Aguila from the command prompt to visualise `discharge.tss`:
```
aguila discharge.tss
```
Note that `discharge.tss` is a text file that you can open in a spreadsheet programme for example. Aguila interprets the text and creates the graphs.

* Which tributary has the highest discharge?

Calibration is beyond the scope of this tutorial, but you can play around with the code to see what happens if you change parameters or variables. For example, what happens with the discharge if we double the precipitation?

< [Prepare the initial section of the model](STREAM_Initial.ipynb) | [Contents](Contents.ipynb) >